# 03-04. cuDF에서 아직 지원하지 않는 기능


## 1. 지원하는 데이터 타입 (cuDF 24.06.00)

cuDF의 `read_csv` 함수는 일부 데이터 타입을 지원하지 않습니다. 예를 들면, `list` 타입이 제대로 처리되지 않는 경우가 있습니다. 


1.  **CSV vs. Parquet**: `read_csv`는 `list`, `struct` 같은 중첩 구조를 지원하지 못하지만, Parquet는 완벽하게 지원합니다. 따라서 **복잡한 데이터엔 Parquet가 유리합니다.**
2.  **Category 타입**: `read_csv`, `read_parquet` 모두 `category` 타입 로드에 제약이 있어, 로드 후 `astype` 변환이 필요할 수 있습니다.
3.  **Decimal 타입**: `decimal`은 JSON 등과 달리 **CSV와 Parquet에서는 잘 지원**됩니다.

## 2. 행(Row) vs 컬럼(Column) 처리

Pandas는 행(Row) 단위로 데이터를 처리하는 경우가 많습니다. 메모리 부족 문제를 해결하기 위해 `chunksize`를 사용해 데이터를 나누고, 각 청크에서 `iterrows()`로 반복 작업을 처리했습니다. 

하지만 **cuDF는 컬럼(Column) 기반 처리가 더 적합하기 때문에 `iterrows()` 메서드를 지원하지 않습니다.** 
이와 관련된 예시 코드는 `03-03-01.OTTO_process_data.ipynb` 파일에서 확인할 수 있습니다. 

데이터 크기, 실행 환경, 코드 구조에 따라 처리 속도는 달라질 수 있습니다. 
예를 들어, 우리 컴퓨터(i9-14900, RTX 4090, RAM 32GB, SSD 256GB, Ubuntu 22.04)에서 실행해보니, `train.jsonl` (11.31GB)을 50개 parquet로 분할 저장에 약 9분이 걸렸습니다.

## 3. cuDF 미지원 기능 요약

정리해보면, 대부분의 Pandas 코드는 cuDF에서도 별다른 수정 없이 동작합니다. 다만, 일부 고급 기능이나 특정 인코딩 지원 등 Pandas의 모든 기능을 cuDF에서 동일하게 사용할 수 없으니, 사용 시 주의하도록 합시다.

| 카테고리 | 명령 | cuDF에서 지원하지 않는 기능 | 비고 |
| :--- | :--- | :--- | :--- |
| **파일 입출력** | `read_csv()` | `encoding='EUC-KR'` | 엑셀에서 utf-8로 저장 |
| | `read_parquet()` | | |
| | `read_json()` | `orient='split'`, `precise_float=True` | 일부 인자 제한 |
| | `to_csv()` | `encoding`, `index_label` | 인코딩 제한 |
| | `to_parquet()` | | |
| | `to_pandas()` | | |
| | `from_pandas()` | | |
| **데이터 탐색** | `info()` | `verbose` 옵션 | 옵션 일부 미지원 |
| | `head()` | | |
| | `tail()` | | |
| | `describe()` | `include='all'`, `percentiles` 매개변수 | 일부 기능 제한 |
| | `shape` | | |
| **인덱싱 및 선택** | `loc` | | |
| | `iloc` | | |
| **데이터 처리** | `unique()` | `sort` 매개변수 | |
| | `groupby()` | `sort` 매개변수, `as_index=False` | `as_index=True` 만 지원 |
| | `sum()` | | |
| | `mean()` | | |
| | `count()` | | |
| | `sort_values()` | `na_position` | NaN 정렬 위치 제어 불가 |
| **반복** | `iterrows()` | **지원 안됨** | 벡터화 연산 또는 `apply_rows()` 대체 |
| **날짜처리** | `DatetimeIndex()` | `strftime()` | `to_datetime()`으로 대체 |
| | `to_datetime()` | | |
| **기타** | `drop()` | `inplace=True` | `inplace` 옵션 미지원 |
| | `style` | **지원 안됨** | `to_pandas()` 로 변환한 후 스타일 적용 |
| | `columns` | | |
| | `index` | | |

